In [103]:
import transformers 
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments,get_scheduler, pipeline, AutoModelForSequenceClassification
from torch import nn, optim 
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
import os
import torch
import datetime
from tqdm import tqdm
import seaborn as sns
import evaluate
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from torch.cuda.amp import autocast, GradScaler

<h1>Load Data and Set for CUDA PARALLEL</h1>
I rented NVIDIA 4090 GPU X 2 for the training and inference for this project.
Belows are code for 1) loading dataset, 2) setting data loader and 3) implementing CUDA NNPARALLEL system.
For the Batch Size, I wanted to use 1024 but due to my GPU memory I used 512 input.

In [48]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
os.environ["TOKENIZERS_PARALLELISM"] = "true"

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6775260134582633173
xla_global_id: -1
]


2024-06-05 19:11:19.754445: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-05 19:11:19.754695: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-05 19:11:19.754844: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required l

In [49]:
model = AutoModelForCausalLM.from_pretrained("heegyu/gpt2-emotion")

In [50]:
tokenizer = AutoTokenizer.from_pretrained('heegyu/gpt2-emotion')


In [51]:
class movRevDataSet(Dataset):
    def __init__(self,root_dir,transform = None):
        self.root_dir = root_dir 
        self.transform = transform
        self.file_list = os.listdir(root_dir)
    def __len__(self):
        return len(self.file_list)
    def __getitem__(self, idx):
        file_name = self.file_list[idx]
        file_path = os.path.join(self.root_dir, file_name)
        try:
            with open(file_path, 'r', encoding='unicode_escape') as file:
                txt = file.read()
        except UnicodeDecodeError as e:
            print(f"Error decoding file {file_path}: {e}")
            raise e
        return txt

In [52]:
trainDataRoot = './aclImdb/train/merged'
testDataRoot = './aclImdb/test/merged'

In [95]:
movRevTrainDS = movRevDataSet(root_dir=trainDataRoot)
movRevTrainDS, movRevValDS = torch.utils.data.random_split(movRevTrainDS,[0.8,0.2])
movRevTestDS = movRevDataSet(root_dir= testDataRoot)
movRevTestDS, movRevTestSubDS = torch.utils.data.random_split(movRevTestDS,[0.9,0.1])

In [96]:

BATCH_SIZE = 2

movRevTrainDL= DataLoader(movRevTrainDS, batch_size = BATCH_SIZE, shuffle= True, num_workers =8 )
movRevValDL = DataLoader(movRevValDS, batch_size =BATCH_SIZE, shuffle = True,num_workers =8 )
movRevTestDL = DataLoader(movRevTestDS, batch_size = BATCH_SIZE, num_workers =8)
movRevTestSubDL = DataLoader(movRevTestSubDS, batch_size = BATCH_SIZE, num_workers =8)

In [55]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [56]:
model = nn.DataParallel(model, device_ids=[0, 1]).cuda()
model.to(device)

DataParallel(
  (module): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-11): 12 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (lm_head): Linear(in_features=768, out_features=50257, bias=False)
  )
)

<h1>FineTune GPT2</h1>


In [57]:
# Tested lr = 2e-5, 2e-4, 2e-3
optimizer = AdamW(model.parameters(),lr=0.001)

#Number of epochs
NUM_EPOCHS = 30

#Input Sequence
MAX_LENGTH = 512
#Total LOOP
NUM_TRAINING = NUM_EPOCHS * len(movRevTrainDL)
# progress_bar = tqdm(range(NUM_TRAINING), leave=True)
lr_scheduler = StepLR(
    optimizer = optimizer,
    step_size = 10,
    gamma=0.1
)


    

In [109]:
def softmax(logits):
    exps = torch.exp(logits - torch.max(logits, dim = -1, keepdim =True).values)
    return exps / torch.sum(exps, dim = -1, keepdim=True)
    
def cross_entropy_loss(logits, targets):
    # logits의 모양: 개단차
    # targets의 모양: 개단

    probs = softmax(logits)
    

    log_probs = torch.log(probs)
    

    nll_loss = -log_probs[range(logits.shape[0]), targets]
    return torch.mean(nll_loss)
    
def my_loss(inputs, logits):

    # logits을 [개*단, 차] 형태로 변형해줘야함. 
    logits = logits.view(-1, logits.size(-1))
    
    # inputs을 [개*단] 형태로 변형 꼭!
    inputs = inputs.view(-1).long()

    loss_value = cross_entropy_loss(logits, inputs)
    
    return loss_value

In [ ]:
model.load_state_dict(torch.load('./model_epoch_4_2024-06-05_08-31-56.pt'))


def train(model, num_epochs, trainDL, valDL, tokenizer, optimizer, scheduler, max_length, patience=5):
    best_val_loss = float('inf')
    epochs_without_improvement = 0

    for epoch in range(num_epochs):
        
        model.train()  
        train_progress_bar = tqdm(trainDL, desc=f"Epoch {epoch + 1} [Train]", leave=True)  # 학습 프로그레스 바 설정
        for texts in train_progress_bar:
            batch_tokenized = tokenizer(texts, max_length=max_length, padding="max_length", truncation=True, return_tensors='pt')
            input_ids = batch_tokenized['input_ids'].to(device)
            attention_mask = batch_tokenized['attention_mask'].to(device)
            labels = batch_tokenized['input_ids'].to(device)  # labels를 input_ids와 동일하게 설정

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = my_loss(input_ids, outputs.logits)

            optimizer.zero_grad()
            loss.sum().backward()
            optimizer.step()
            scheduler.step()

        
        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        torch.save(model.state_dict(), f"./model_epoch_{epoch + 1}_{current_time.replace(' ', '_').replace(':', '-')}.pt")
        
        # Eval
        model.eval()
        total_val_loss = 0
        val_progress_bar = tqdm(valDL, desc=f"Epoch {epoch + 1} [Validation]", leave=True)  # 검증 프로그레스 바 설정
        with torch.no_grad():
            for texts in val_progress_bar:
                batch_tokenized = tokenizer(texts, max_length=max_length, padding="max_length", truncation=True, return_tensors='pt')
                input_ids = batch_tokenized['input_ids'].to(device)
                attention_mask = batch_tokenized['attention_mask'].to(device)
                labels = batch_tokenized['input_ids'].to(device)  # labels를 input_ids와 동일하게 설정

                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                val_loss = my_loss(input_ids, outputs.logits)
                total_val_loss += val_loss.sum().item()

    
        total_val_loss /= len(valDL)
       
      # Early Stop
        if total_val_loss < best_val_loss:
            best_val_loss = total_val_loss
            epochs_without_improvement = 0
            torch.save(model.state_dict(), f"./best_model_{current_time.replace(' ', '_').replace(':', '-')}.pt")
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print(f"Validation loss has not improved for {patience} epochs. Stopping training.")
                break

train(model, NUM_EPOCHS, movRevTrainDL, movRevValDL, tokenizer, optimizer, lr_scheduler, MAX_LENGTH)



<h1>Generate 30 Reviews<h1>

In [ ]:
model.load_state_dict(torch.load('./model_epoch_7_2024-06-05_15-56-39.pt'))
prompt_1 = [
    "The movie was",
    "The lead actor's performance was",
    "The cinematography in the film was",
    "The plot of the movie was",
    "The supporting cast's performances were",
    "The soundtrack of the film was",
    "The direction of the movie was",
    "The special effects in the film were",
    "The pacing of the movie was",
    "The dialogue in the film was",
    "The film's setting was",
    "The movie's themes were",
    "The opening scene was",
    "The ending of the movie was",
    "The costumes and makeup were",
    "The movie's humor was",
    "The emotional impact of the film was",
    "The character development was",
    "The movie's message was",
    "The action scenes were",
    "The film's visual style was",
    "The editing of the movie was",
    "The movie's rewatch value is",
    "The film's cultural impact was",
    "The chemistry between the actors was",
    "The movie's originality was",
    "The film's use of symbolism was",
    "The movie's suspense level was",
    "The realism in the film was",
    "The overall experience of watching the movie was"
]

gen_text = []
for text in prompt_1:
    tokenized = tokenizer(text, return_tensors="pt").input_ids.to(device)
    gen_token = model.module.generate(
        input_ids=tokenized, 
        min_length=50, 
        max_length=100,
        no_repeat_ngram_size=3,
        temperature=1,
        num_beams=5,
        top_p = 0.99
    )
    
    pred = tokenizer.batch_decode(gen_token, skip_special_tokens=True)[0]
    gen_text.append(pred)
    print("생성토큰: ",pred)


    

In [ ]:
gen_text = [
    "The movie was really good. I'm glad that I'm not the only one who has seen this film, but I think it's the best movie ever. The main problem with this film is that it's not a good movie, but it's a very good movie.",
    "The lead actor's performance was excellent. The dialogue is excellent, but the direction and direction of the film is not as good as it was in the film. The main problem with this film is the fact that it lacks a lot of dialogue. The plot is not quite good.",
    'The cinematography in the film was excellent. The dialogue is excellent, but the dialogue is not as good as it was in the movie. The main problem with this film is the fact that it is not a good film. There is a lot of fun to be done with it.',
    'The plot of the movie was simple and simple. The main problem with this movie is that it suffers from a flawed plot. The plot is flawed. The dialogue is flawed, the direction of the film is flawed in terms of direction. The direction of this film is not as good as it was in the original film. There is a lot of potential for this film to be flawed.',
    "The supporting cast's performances were excellent. The dialogue is excellent, but the direction and direction of the film is not as good as it was in the film. The main problem with this film is the fact that it lacks a lot of dialogue. The fact that the main problem is that it's not as much of a problem as it is in the movie.",
    'The soundtrack of the film was excellent. The dialogue is excellent, but the dialogue is not as good as it was in the film. The main problem with this film is the fact that it lacks a lot of dialogue. The fact is that it is not a good film.',
    'The direction of the movie was not quite good, but it was better than it was supposed to be. The main problem with this film is that it is not quite as good as it could be. I think it is a lot of fun to watch.',
    "The special effects in the film were excellent. The film is not quite as good as it was in the original film. The main problem with this film is that it's not quite like the original movie. The dialogue is not particularly good. The movie is not as good, but it is not much better than the original.",
    "The pacing of the movie was excellent. The dialogue is excellent, but the direction of the film is not as good as it was in the original film. The main problem with this film is that it is not a good film, but it is a very good film. I think it's not a bad film. If you're looking at this film, you'll be disappointed in the direction and direction of its direction.",
    "The dialogue in the film was excellent. The dialogue is excellent, but the dialogue is not as good as it was in the movie. The main problem with this film is the fact that it is not a good film. The fact is that it's not an excellent film.",
    "The film's setting was excellent. The dialogue is excellent, but the dialogue is not as good as it was in the film. The main problem with this film is the fact that it is not a good movie. The fact is that it's not a bad movie.",
    "The movie's themes were simple and simple. The main problem with this film is that it is not a simple film, but it is a very good one. The plot is simple, but the main problem is not simple. It is a simple movie, but I think it deserves to be considered as a classic.",
    'The opening scene was excellent. The dialogue is excellent, but the direction and direction of the film is not as good as it was in the film. The main problem with this film is the fact that it is not a good film, but it is a very good film.',
    'The ending of the movie was excellent. The dialogue is excellent, but the direction and direction of the film is not as good as it was in the film. The main problem with this film is the fact that it is not a good film. There is a lot of fun to be done with it.',
    'The costumes and makeup were excellent, though they could have been better. The main problem with this film is that it lacks a lot of potential. The film is not particularly good. The dialogue is not quite good, but it could be better. A lot of fun to watch.',
    "The movie's humor was excellent, though it was not quite as good as it could be. The dialogue is excellent, but it wasn't quite as funny as it was. The main problem with the film is the fact that it's not really funny.",
    'The emotional impact of the film was apparent in the film, but it was not as much of a film as I think it could have been better than the film. The main problem with this film is that it lacks the emotional and emotional effects of the movie.',
    "The character development was excellent. The dialogue is excellent, but the dialogue is not as good as it was in the film. The main problem with this film is the fact that it is not a good movie. The fact is that it's not a bad movie.",
    "The movie's message was simple and simple. The main problem with this film is that it's not really about the movie. It's not about the film, but it's about the plot. The plot is simple, but the main problem is the fact that it wasn't about the main plot. It was about the direction of the film.",
    'The action scenes were excellent. The dialogue is excellent, but the direction and direction of the film is not as good as it was in the film. The main problem with this film is the fact that it is not a good film, but it is a very good film.',
    "The film's visual style was excellent. The dialogue is excellent, but the dialogue is not as good as it was in the film. The main problem with this film is the fact that it is not a good film, but it is a very good film. There is a lot of fun to be done with it. The fact is that it's not a great film.",
    'The editing of the movie was excellent. The dialogue is excellent, but the dialogue is not as good as it was in the original film. The main problem is that it is not a good movie. The plot is not quite good. The direction of the film is not very good.',
    "The movie's rewatch value is one of the best things I've ever seen in a movie. I'm glad that I'm not the only one who has seen this movie, but I think it's the best thing I've seen in the movie.",
    "The film's cultural impact was not as much as it could have been. The main problem is the fact that it is not a film that I want to see again. The film is not about the film, but it is about the role of the director.",
    'The chemistry between the actors was excellent. The chemistry between them is excellent, but the chemistry is not as good as the chemistry between their roles. The dialogue is excellent and the dialogue is not quite as good. The direction of the movie is not particularly good, but it is not much better than the chemistry of the film. The main problem with this film is that it lacks a lot of potential.',
    "The movie's originality was restored to the original film, but it was not restored to its original film. The original film was restored from the original movie, and it is not restored. The main problem with this film is the fact that it's not restored, but the main problem is that it lacks a lot of originality.",
    "The film's use of symbolism was not particularly good, but it was a very good film. I think it could have been better, but I think that it could be better, if it hadn't been so much of an effort. The main problem with this film is the fact that it's not quite as good as it was in the first place.",
    "The movie's suspense level was surprisingly low, but it was surprisingly good. I think it was a very good movie. I'm glad that it's not as much of a suspenseful movie as it was. The main problem is that it wasn't much of an suspenseful one.",
    'The realism in the film was excellent. The dialogue is excellent, but the dialogue is not as good as it was in the movie. The main problem with this film is the fact that it is not a realistic film. There is a lot of fun to be had in this film.',
    "The overall experience of watching the movie was excellent, but it was not quite as good as it could be. The film is not quite like it was. The dialogue is not particularly good, though it was excellent. The main problem is the fact that it wasn't quite as much fun as it would be if it hadn't been a fun movie. The fact is that it's not a fun film."
]

<h1>Bleu Score Computation: Result: 0.02008190923851741</h1>

In [97]:
## os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# # Load model and set to evaluation mode
model.load_state_dict(torch.load('./model_epoch_7_2024-06-05_15-56-39.pt'))
model.eval()

def test(model, testDL, max_length):
    pred = []
    ground = []
  
    for text in tqdm(testDL, desc="Testing Progress"):
        # To Use 10 word
        text = text[0]
        prompt = " ".join(text.split()[:10])
        temp_ground = " ".join(text.split()[10:])
        ground.append([temp_ground])
        
        # Tokenize Prompt
        tokenized = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
      
        ground_tokens = tokenizer.encode(temp_ground, add_special_tokens=False)
        target_length = len(ground_tokens)
       
        print(f"Prompt: {prompt}")
        print(f"Tokenized Prompt Shape: {tokenized.shape}")
        # print(f"Ground Truth Tokens: {ground_tokens}")
        
        try:
            # gen_token = model.module.generate(
            #     input_ids=tokenized, 
            
            #     max_length=max_length,
            #     # no_repeat_ngram_size=3,
            #     # num_return_sequences=1,  # 한 개의 시퀀스만 생성
            #     # no_repeat_ngram_size=2,  # 반복 방지를 위해 N그램 설정
            #     # early_stopping=True      # 조기 종료를 사용
            # )
            gen_token = model.module.generate(
                input_ids=tokenized, 
                min_length=300,    
                max_length=500,
                # num_return_sequences=1,  
                no_repeat_ngram_size=3,  
                # # early_stopping=True,    
                # num_beams=5,             
                # temperature=0.5,         
                # top_k=50,                
                # top_p=0.95               
            )
            print(f"Generated Tokens Shape: {gen_token.shape}")
        except RuntimeError as e:
            print(f"Error during generation: {e}")
            print(f"Tokenized input: {tokenized}")
            continue
        
        # Decode the Generated Text
        temp_pred = tokenizer.batch_decode(gen_token, skip_special_tokens=True)[0]
        # Comparation start from the 11th word. 
        temp_pred = " ".join(temp_pred.split()[10:])
        
        # Pred should be wrapped by list
        pred.append(temp_pred)
    print("Sample Predictions:")
    for i in range(min(5, len(pred))):
        print(f"Pred: {pred[i]}")
        print(f"Ground: {ground[i][0]}\n")
        
    bleu = evaluate.load("bleu")
    results = bleu.compute(predictions=pred, references=ground)
    print(results)


test(model, movRevTestSubDL, 500)

Testing Progress:   0%|                                | 0/1250 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Two popular actors are paired in showtime like De Niro
Tokenized Prompt Shape: torch.Size([1, 12])


Testing Progress:   0%|                        | 1/1250 [00:02<53:16,  2.56s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Tokens Shape: torch.Size([1, 349])
Prompt: Prue and Piper bring Dr. Griffiths to their home to
Tokenized Prompt Shape: torch.Size([1, 13])


Testing Progress:   0%|                        | 2/1250 [00:04<43:59,  2.12s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Testing Progress: 100%|████████████████████▉| 1249/1250 [38:43<00:02,  2.02s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Tokens Shape: torch.Size([1, 401])
Prompt: I'M BOUT IT(1997)<br /><br />Developed & published by No Limit
Tokenized Prompt Shape: torch.Size([1, 22])


Testing Progress: 100%|█████████████████████| 1250/1250 [38:45<00:00,  1.86s/it]

Generated Tokens Shape: torch.Size([1, 302])
Sample Predictions:
Pred: and the movie. The roles are not quite as good as the movie, but they are not as good. The dialogue is not quite good. I think it's a good movie. I'm not really sure it's the best. The characters are not particularly good. There is a lot of potential to be lost in this. The movie is not really good. It is not a good film. I don't think it is a good one. The main problem is the fact that it is not good. A lot of fun. The film is not particularly fun. I want to see it. The plot is not as fun. It's not a fun movie. It was not a very fun movie, though it was a fun one. I can assure you that it was not quite fun. A good one, but it was quite a fun and I think the main problem with the movie is the direction of the film. The direction of this film is the main issue. The fact that the direction is not very good. In fact, it is the most fun and the film is to watch. The action is not much fun. There are some of the best rol

{'bleu': 0.02008190923851741, 'precisions': [0.24351890858420558, 0.04634772854131329, 0.007895423699321386, 0.001825086343770972], 'brevity_penalty': 1.0, 'length_ratio': 1.1171453826358992, 'translation_length': 370308, 'reference_length': 331477}


<h2>BERT SENTIMENT CLASSIFICATION & Analysis and Visulation </h2>
All the needed explanation is in submitted report

In [43]:
pipe2 = pipeline("text-classification", model="textattack/bert-base-uncased-SST-2")
tokenizer2 = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-SST-2")
model2 = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-SST-2")

In [114]:

tokenizer2 = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-SST-2")
model2 = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-SST-2")

classifier = pipeline('sentiment-analysis', model=model2, tokenizer=tokenizer2)





results = classifier(gen_text)
print(results)
# Convert results to labels
labels = ["LABEL_1" if result['label'] == 'LABEL_1' else "LABEL_0" for result in results]


true_labels = [
    "LABEL_1", "LABEL_0", "LABEL_0", "LABEL_0", "LABEL_0", "LABEL_0", "LABEL_0",
    "LABEL_0", "LABEL_0", "LABEL_0", "LABEL_0", "LABEL_0", "LABEL_1", "LABEL_0",
    "LABEL_0", "LABEL_0", "LABEL_0", "LABEL_0", "LABEL_0", "LABEL_1", "LABEL_0",
    "LABEL_1", "LABEL_0", "LABEL_0", "LABEL_0", "LABEL_0", "LABEL_0", "LABEL_0",
    "LABEL_0", "LABEL_0"
]

accuracy = accuracy_score(true_labels, labels)
print(f"Accuracy: {accuracy}")

pos = 0 
neg = 0

for text, label, true in zip(gen_text, labels, true_labels):
    if(label == "LABEL_1") :
        pos = pos +1
    else: neg = neg +1
   
    if(label != true):
        print(f"Text: {text}\nLabel: {label}\nTrue:{true}")
print("POS",pos,"NEG",neg)

[{'label': 'LABEL_1', 'score': 0.9986995458602905}, {'label': 'LABEL_0', 'score': 0.9965370893478394}, {'label': 'LABEL_0', 'score': 0.9843654632568359}, {'label': 'LABEL_0', 'score': 0.9980173110961914}, {'label': 'LABEL_0', 'score': 0.9946562051773071}, {'label': 'LABEL_0', 'score': 0.9970692992210388}, {'label': 'LABEL_0', 'score': 0.6378469467163086}, {'label': 'LABEL_0', 'score': 0.9979678988456726}, {'label': 'LABEL_0', 'score': 0.8281629085540771}, {'label': 'LABEL_0', 'score': 0.9966611862182617}, {'label': 'LABEL_0', 'score': 0.9566648006439209}, {'label': 'LABEL_1', 'score': 0.9935171604156494}, {'label': 'LABEL_1', 'score': 0.6264100670814514}, {'label': 'LABEL_0', 'score': 0.9838851690292358}, {'label': 'LABEL_0', 'score': 0.7056890726089478}, {'label': 'LABEL_0', 'score': 0.9973963499069214}, {'label': 'LABEL_0', 'score': 0.9987564086914062}, {'label': 'LABEL_0', 'score': 0.9559622406959534}, {'label': 'LABEL_0', 'score': 0.9972262978553772}, {'label': 'LABEL_1', 'score': 